# Migration - Local Area

-----

### Requirements



#### Observations & Dimensions

We're taking 5 tabs (see below table). The `observations` and the `indictor` will vary a little for each tab,

|Tab name| Get obs from columns named | Set 'Indictor' dimension to |
|---|---|---|
| Short Term Migration Flow | Short-Term International Migration Inflow Estimates | Inflow Estimates
| Non-UK Born Population | Non-UK Born Estimate | Not UK Born Population
| Non British Population | Non-British Estimate |  Not British Population
| NINo Registrations | Migrant NINo Registrations | NINo Registrations
| GP Registrations | New Migrant GP Registrations | New Gp Registrations


The required dimensions are:

* **Geography** - use the area code column
* **Time** - from the top and just the year, so "Jan 2009 to Dec 2009" == 2009
* **Indictor** - as per above table

-----
    
Notes:

It's always worth getting the file out of /sources and having a look over.

In [1]:
new_indicators = {"Short-Term Migration Inflows": "Inflow Estimates",
                  "Non-UK Born Population": "Not UK Born Population",
                  "Non-British Population": "Not British Population",
                  "NINo Registrations": "NINo Registrations",
                  "GP Registrations": "New GP registrations"}

In [2]:
%cd mock-transformations/

/workspace/mock-transformations


In [3]:
from databaker.framework import *
import pandas as pd

# the Seasonally Adjusted Tabs
sa_tabs = loadxlstabs("./sources/lamisspreadsheet.xls")

Loading ./sources/lamisspreadsheet.xls which has size 1138688 bytes
Table names: ['Notes', 'Migration Flows', 'Short-Term Migration Inflows', 'Non-UK Born Population', 'Non-British Population', 'NINo Registrations', 'GP Registrations', 'Births to Non-UK Born Mothers']


In [4]:
# This is the test I need to do to see how merged cells behave

sa_tabs[2].excel_ref("C1").expand(RIGHT).is_not_blank()

# So it looks like this is a case of CLOSEST, ABOVE, and will need to substring after the fact in pandas
# Turns out it's closest left

{<Q1 'Mid-2015 to Mid-2016'>, <M1 'Mid-2013 to Mid-2014'>, <I1 'Mid-2011 to Mid-2012'>, <G1 'Mid-2010 to Mid-2011'>, <E1 'Mid-2009 to Mid-2010 '>, <S1 'Mid-2016 to Mid-2017'>, <O1 'Mid-2014 to Mid-2015'>, <K1 'Mid-2012 to Mid-2013'>, <C1 'Mid-2008 to Mid-2009 '>}

In [5]:
work = []

In [6]:
for tab in sa_tabs:
    if tab.name.split()[-1] in ("Inflows", "Registrations"):
        print("😍 " + tab.name)
        indicator = tab.excel_ref("C2").expand(RIGHT).filter(matches_regexp("(Inflow)|(Registrations)"))
    elif tab.name.split()[-1] == "Population":
        print("🥰 " + tab.name)
        indicator = tab.excel_ref("C3").expand(RIGHT).filter(matches_regexp("Estimate"))
    else:
        # print("😖 " + tab.name)
        continue

    time = tab.excel_ref("C1").expand(RIGHT).is_not_blank()
    geography = tab.excel_ref("A4").expand(DOWN).is_not_blank()
    observations = geography.waffle(indicator)

    dimensions = [HDim(geography, "Geography", DIRECTLY, LEFT),
                  HDim(time, "Year", CLOSEST, LEFT),
                  HDim(indicator, "Indicator", DIRECTLY, ABOVE)]

    df = ConversionSegment(tab, dimensions, observations).topandas()
    df["Indicator"] = new_indicators[tab.name]

    work.append(df)

😍 Short-Term Migration Inflows

🥰 Non-UK Born Population

🥰 Non-British Population

😍 NINo Registrations

😍 GP Registrations



In [7]:
output = pd.concat(work)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
df["Year"] = df["Year"].str[4:8].astype(int)

In [9]:
df.drop(columns="DATAMARKER", inplace=True)

In [10]:
df

,OBS,Geography,Year,Indicator
0,577566,E92000001,2008,New GP registrations
1,604357,E92000001,2009,New GP registrations
2,613124,E92000001,2010,New GP registrations
3,578105,E92000001,2011,New GP registrations
4,587279,E92000001,2012,New GP registrations
...,...,...,...,...
4265,3252,E06000014,2013,New GP registrations
4266,3018,E06000014,2014,New GP registrations
4267,2918,E06000014,2015,New GP registrations
4268,3061,E06000014,2016,New GP registrations
